In [16]:
from server import *
import pdb
import openai
import re


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [ ]:
url = 'https://www.foodnetwork.com/recipes/rachael-ray/meatloaf-muffins-with-barbecue-sauce-recipe-2118102'
text = scrape_text(url)

In [37]:
url = 'https://therealfooddietitians.com/bbq-meatloaf-muffins/'

In [ ]:
# Prompt Goals
# - Determine if a recipe is present on the page?
# - Extract the recipe
# - Extract the number of servings
# - Extract the instructions
# - Extract the recipe title
# - Extract nutritional information
# - Extract summary of blog post? or dietary restrictions
# - site the website

In [38]:
def create_prompt(site_text):
    prompt = """Determine if a recipe is contained in the text and if there is a recipe, follow these instructions:
    Extract the recipe title.
    Extract the Ingredients and portions of each.
    Extract the instructions.
    Extract the number of servings.
    Extract the time it will take to make if mentioned.
    Extract the time it will take to cook if mentioned.
    Store this information in json.

    Output the result in this format:  
    {
        'recipes': [
            {'name': 'recipe name goes here', 
             'ingredients' : [
                "amount - ingredient",
                "amount - ingredient"
                ],
             'instructions' : [
             "Instruction goes here",
             "Instruction goes here"
                ],
             'serving size': 'number of servings goes here',
             'citation' : 'citation goes here'
        ]
    }


    '"""
    
def gpt_request(prompt):
    openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=
        [
            {'role':'system', 'content': 'Your job is to extract useful information from text'},
            {'role' : 'user', 'content': prompt}
        ]
    )

In [39]:
text = scrape_text(url)
text

'\n\n\nJust a moment...\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        Enable JavaScript and cookies to continue\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n'

In [40]:
len(text)

135

In [41]:
response = requests.get(url)

In [43]:
soup = BeautifulSoup(response.content, 'html.parser')



In [57]:
url = "https://therealfooddietitians.com/bbq-meatloaf-muffins/"  # replace with the URL of the website you want to scrape
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

articles = soup.find_all()
articles

[<html lang="en-US">
 <head>
 <title>Just a moment...</title>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
 <meta content="noindex,nofollow" name="robots"/>
 <meta content="width=device-width,initial-scale=1" name="viewport"/>
 <link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"/>
 </head>
 <body class="no-js">
 <div class="main-wrapper" role="main">
 <div class="main-content">
 <noscript>
 <div id="challenge-error-title">
 <div class="h2">
 <span class="icon-wrapper">
 <div class="heading-icon warning-icon"></div>
 </span>
 <span id="challenge-error-text">
                         Enable JavaScript and cookies to continue
                     </span>
 </div>
 </div>
 </noscript>
 <div id="trk_jschal_js" style="display:none;background-image:url('/cdn-cgi/images/trace/managed/nojs/transparent.gif?ray=7becdaa0fb13155a')"></div>
 <form action="/bbq-meatloaf-muffins/?__cf_chl_f_tk=N6wFG2GZJzxAA6Bgt0.u

In [32]:
# Find all article tags and concatenate their text
article_text = ""
for article in soup.find_all('article'):
    article_text += article.get_text().strip() + " "

# Print the concatenated text
print(article_text)


''

In [15]:
soup = BeautifulSoup(response.content, 'html.parser')

# Find all div tags in the HTML
div_tags = soup.find_all('div')

# Find all the tags to exclude from text extraction
tags_to_exclude = ['nav', 'menu', 'footer']

# Loop through each tag to exclude and remove it from the soup object
for tag in tags_to_exclude:
    for el in soup.find_all(tag):
        el.extract()

# Extract all the remaining text from the page
text = soup.get_text()

text.split()

print(len(text))
# Print the extracted text
print(text)


11195

























BBQ Meatloaf Muffins with Sweet Potato Topping - The Real Food Dietitians
















 




















































 Skip to primary navigation Skip to main content Skip to primary sidebar




New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW




















Main MenuThe Real Food DietitiansEat well. Live well. Be well.
Search...
Display Search Bar


Home  BlogBBQ Meatloaf Muffins with Sweet Potato Topping


W
Whole30

EF
Egg-Free

GF
Gluten-free

NF
Nut-free

GR
Grain-Free

DF
Dairy-Free

P
Paleo






 30 mins


4.9
/5



70 Comments




 Jump to Recipe




 




 












By: Jessica Beacom
Updated: 4/1/20






This post may contain affiliate links. Please read our disclosure policy.





What do you get when dinner and dessert have a baby? You get BBQ Meatloaf Muffins with Sweet Potato Topping that are almost too cute to eat.

Small in size but mighty in flavor
Meatloaf regularly finds its way on my weekly mea

In [ ]:
# Attempting to store the text from different div tags on the page in dictionaries and then parsing the one i want based on 
# what text is in them. ie. this region of text contains the keywords 'ingredients', 'prep', cooking time', so this must have the recipe!

In [24]:
# Create a BeautifulSoup object with the page content
soup = BeautifulSoup(response.content, 'html.parser')

# Define a list of keywords to search for within the text
keywords = ['Ingredients', 'Preparation', 'Cooking time', 'Serving size']

# Search for all <div> tags on the page
divs = soup.find_all('div')

# Create a dictionary to store the score for each <div> tag
scores = {}

# Loop through each <div> tag and calculate its score
for div in divs:
    # Check if the <div> tag is a child of <nav>, <menu>, or <footer>
    if div.parent.name in ['nav', 'menu', 'footer']:
        continue
    # Otherwise, calculate the score as before
    text = div.text
    score = 0
    for keyword in keywords:
        if keyword in text:
            score += 1
    scores[div] = score

# Find the <div> tag with the highest score
highest_score = max(scores.values())
highest_div = None
for div, score in scores.items():
    if score == highest_score:
        highest_div = div
        break

# Extract the text from the <div> tag with the highest score
recipe_text = highest_div.text.strip()

print(len(recipe_text))
# Print the recipe text
print(recipe_text)


11492
Skip to primary navigation Skip to main content Skip to primary sidebar




New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW




New? Start here!
About
Shop
Contact
















Main MenuThe Real Food DietitiansEat well. Live well. Be well.Free Meal Plans
Recipes

All Recipes
Meal

Breakfast
Lunch & Dinner
Snacks
Salads & Sides
Soups & Chilis
Appetizers
Drinks & Smoothies
Desserts
Condiments


Diet

Whole30
Gluten-free
Grain-Free
Dairy-Free
Paleo
Egg-Free
Nut-free
Vegetarian
Vegan


Method

Instant Pot
Slow Cooker
Grill
Stovetop
Oven
Blender
Air Fryer
Smoker
No-Bake
Meal Prep


Protein

Chicken
Turkey
Beef
Pork
Fish & Seafood
Eggs
Meatless


Filter Recipes


Wellness

What is Real Food?
Free Meal Plans
Wellness


Cookbook





 Stay Updated On New Posts






Stay updated on our latest posts and more!
Please enable JavaScript in your browser to complete this form.NameEmail *GDPR Consent *I consent to receive emails from The Real Food Dietitians *Sign me up! 






Search

In [17]:
# Attempt to retun the text before the © symbol but only after we've found 'ingredients' and 'instructions'

In [18]:
# text = "This is a blog post about a recipe. First, let's talk about the ingredients. Here's what you'll need: flour, sugar, eggs, milk. Next, the instructions: preheat the oven, mix the ingredients, and bake for 30 minutes. © All rights reserved 2023"

# Search for the keywords 'ingredients' and 'instructions' and the occurrence of at least 5 numbers
match = re.search(r'(ingredients|instructions).*?\d.*?©', recipe_text, flags=re.IGNORECASE|re.DOTALL)

if match:
    # Get the index of the © symbol
    index = match.start(0) + match.group(0).rindex('©')
    # Extract the text before the © symbol
    result = text[:index]
    print(len(result))
    print(result)
else:
    print("Text not found")


7985

























BBQ Meatloaf Muffins with Sweet Potato Topping - The Real Food Dietitians
















 




















































 Skip to primary navigation Skip to main content Skip to primary sidebar




New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW




















Main MenuThe Real Food DietitiansEat well. Live well. Be well.
Search...
Display Search Bar


Home  BlogBBQ Meatloaf Muffins with Sweet Potato Topping


W
Whole30

EF
Egg-Free

GF
Gluten-free

NF
Nut-free

GR
Grain-Free

DF
Dairy-Free

P
Paleo






 30 mins


4.9
/5



70 Comments




 Jump to Recipe




 




 












By: Jessica Beacom
Updated: 4/1/20






This post may contain affiliate links. Please read our disclosure policy.





What do you get when dinner and dessert have a baby? You get BBQ Meatloaf Muffins with Sweet Potato Topping that are almost too cute to eat.

Small in size but mighty in flavor
Meatloaf regularly finds its way on my weekly meal

In [25]:
def get_first_h1_tag(soup):
    h1_tag = soup.find('h1')
    return h1_tag.text.strip() if h1_tag else "No title found"


In [26]:
title = get_first_h1_tag(soup)
print(title)

BBQ Meatloaf Muffins with Sweet Potato Topping


In [ ]:
### Use booelans instead of regex to find the © symbol and slice the text to return the part that I want.

In [27]:
def extract_recipe_text(text):
    # Split the text into individual words
    words = text.split()

    # Initialize variables to keep track of state
    found_ingredients = False
    found_instructions = False
    num_count = 0
    ignore = False

    # Loop through each word in the text
    for i, word in enumerate(words):
        # Check if the word is 'ingredients' or 'instructions'
        if word.lower() == 'ingredients':
            found_ingredients = True
        elif word.lower() == 'instructions':
            found_instructions = True

        # Check if the word contains a digit
        if any(char.isdigit() for char in word):
            num_count += 1

        # Check if we've seen 5 digits and found the keywords
        if num_count >= 5 and found_ingredients and found_instructions:
            # Look for the © symbol and stop appending text
            if '©' in word:
                ignore = True
            break

    # Get the recipe text by slicing the words up to the index where the © was found
    recipe_text = ' '.join(words[:i]) if ignore else ' '.join(words)

    # Strip leading and trailing whitespace from the recipe text
    return recipe_text.strip()


In [28]:
text = extract_recipe_text(recipe_text)
print(len(text))
print(text)

10663
Skip to primary navigation Skip to main content Skip to primary sidebar New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New? Start here! About Shop Contact Main MenuThe Real Food DietitiansEat well. Live well. Be well.Free Meal Plans Recipes All Recipes Meal Breakfast Lunch & Dinner Snacks Salads & Sides Soups & Chilis Appetizers Drinks & Smoothies Desserts Condiments Diet Whole30 Gluten-free Grain-Free Dairy-Free Paleo Egg-Free Nut-free Vegetarian Vegan Method Instant Pot Slow Cooker Grill Stovetop Oven Blender Air Fryer Smoker No-Bake Meal Prep Protein Chicken Turkey Beef Pork Fish & Seafood Eggs Meatless Filter Recipes Wellness What is Real Food? Free Meal Plans Wellness Cookbook Stay Updated On New Posts Stay updated on our latest posts and more! Please enable JavaScript in your browser to complete this form.NameEmail *GDPR Consent *I consent to receive emails from The Real Food Dietitians *Sign me up! Search... Display Search BarFree Meal Plans Meal Prep Breakfasts Ma

In [16]:
close_recipe_index = div_text.find('recipe')
snippet = div_text[close_recipe_index:] # Get everything after Recipe
# Check if the word ingredients and instructions are found soon after. If not then you have the wrong 'recipe'

# find the word ingredient

# find the word instructions

# find reviews by 'comment', or 'leave a comment', or 'share', 'recipe by', 'posted on', or 'reviews' or 'rating' (and make sure they occur after the indexes for ingredients and instructions)


In [34]:
div_text

'Skip to primary navigation Skip to main content Skip to primary sidebar Skip to primary navigation Skip to primary navigation Skip to main content Skip to main content Skip to primary sidebar Skip to primary sidebar New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW\n\n\n\n\nNew? Start here!\nAbout\nShop\nContact New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW\n\n\n\n\nNew? Start here!\nAbout\nShop\nContact New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New!Get Your 30 Low-Carb High-Protein Recipe eBook NOW New!  New? Start here!\nAbout\nShop\nContact New? Start here!\nAbout\nShop\nContact New? Start here!\nAbout\nShop\nContact New? Start here!\nAbout\nShop\nContact New? Start here! New? Start here! New? Start here! About About About Shop Shop Shop Contact Contact Contact          Main MenuThe Real Food 

In [27]:
list_parents = soup.find_all(['ol', 'ul'])#, class_=lambda x: x and ('nav' not in x.lower() and 'footer' not in x.lower()))

# Extract the text content of those elements
for parent in list_parents:
    text = parent.get_text().strip()
    print(text)

Skip to primary navigation Skip to main content Skip to primary sidebar
New? Start here!
About
Shop
Contact
Free Meal Plans
Recipes

All Recipes
Meal

Breakfast
Lunch & Dinner
Snacks
Salads & Sides
Soups & Chilis
Appetizers
Drinks & Smoothies
Desserts
Condiments


Diet

Whole30
Gluten-free
Grain-Free
Dairy-Free
Paleo
Egg-Free
Nut-free
Vegetarian
Vegan


Method

Instant Pot
Slow Cooker
Grill
Stovetop
Oven
Blender
Air Fryer
Smoker
No-Bake
Meal Prep


Protein

Chicken
Turkey
Beef
Pork
Fish & Seafood
Eggs
Meatless


Filter Recipes


Wellness

What is Real Food?
Free Meal Plans
Wellness


Cookbook





 Stay Updated On New Posts






Stay updated on our latest posts and more!
Please enable JavaScript in your browser to complete this form.NameEmail *GDPR Consent *I consent to receive emails from The Real Food Dietitians *Sign me up!
All Recipes
Meal

Breakfast
Lunch & Dinner
Snacks
Salads & Sides
Soups & Chilis
Appetizers
Drinks & Smoothies
Desserts
Condiments


Diet

Whole30
Gluten-free
Gr

In [19]:
# find all the <ol> and <ul> tags in the HTLML document
list_tags = soup.find_all(['ol', 'ul'])

In [ ]:
# Iterate through each <ol> <ul> tag and find their parent <div>
for list_tag in list_tags:
    parent_div = list_tag.find_parent('div')
    print(parent_div)

In [23]:
for ul in soup.find_all(['ul', 'ol']):
    parent_div = ul.find_parent('div', class_=lambda x: x not in ['nav', 'menu', 'footer'])
    if parent_div:
        print(parent_div)

<div class="site-container"><ul class="genesis-skip-link"><li><a class="screen-reader-shortcut" href="#genesis-nav-primary"> Skip to primary navigation</a></li><li><a class="screen-reader-shortcut" href="#genesis-content"> Skip to main content</a></li><li><a class="screen-reader-shortcut" href="#genesis-sidebar-primary"> Skip to primary sidebar</a></li></ul><div class="before-header">
<div class="wrap">
<div class="featured-content sm-col-1 cta-header cta-header-desktop hidden-on-mobile">
<div class="wrap">
<div class="section-content">
<span class="more-link-container"><a class="more-link" href="https://therealfoodrds.ck.page/products/30-low-carb-high-protein-recipes" rel="noopener" target="_blank"><em>New!</em>Get Your 30 Low-Carb High-Protein Recipe eBook NOW<span class="icon-font icon-arrow"></span></a></span>
</div>
</div>
</div>
<div class="nav-and-social hidden-on-mobile">
<nav class="nav-header"><div class="wrap"><ul class="menu genesis-nav-menu js-superfish limit-depth-1" id="